In [1]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import RadioButtons
from IPython.display import display, clear_output
import scipy.signal
from scipy.signal import upfirdn, welch, decimate, convolve, firwin2, lfilter
from scipy.special import erfc
from ipywidgets import Checkbox, Button, Output, VBox, HBox, Dropdown, Layout
from commpy.channels import awgn
import time
print("Libraries added successfully!")

Libraries added successfully!


In [2]:
# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html3 = widgets.HTML(
  value=loading
)
timer_html3 = widgets.HTML(
    value="Elapsed time: - seconds"
)

# Define the modulation schemes and levels
modulation_schemes = {
    'ASK': ['2-ASK', '4-ASK', '8-ASK', '16-ASK', '32-ASK', '64-ASK'],
    'QAM': ['4-QAM', '16-QAM', '64-QAM'],
    'PSK': ['2-PSK', '4-PSK', '8-PSK'],
    'FSK': ['2-FSK', '4-FSK', '8-FSK', '16-FSK', '32-FSK'],
    'MSK': ['MSK (with precoding)', 'MSK (without precoding)']
}

# Dropdown for modulation schemes
modulation_dropdown = widgets.Dropdown(
    options=list(modulation_schemes.keys()),
    value='ASK',
    description='Modulation:'
)

# Dropdown for levels
levels_dropdown = widgets.Dropdown(
    description='Levels:'
)


def update_levels_dropdown(*args):
    levels_dropdown.options = modulation_schemes[modulation_dropdown.value]
    levels_dropdown.value = levels_dropdown.options[0]  # Set default value


modulation_dropdown.observe(update_levels_dropdown, 'value')
update_levels_dropdown()  # Initialize the levels dropdown

plot_output = widgets.Output()


def ask_errors(k, M, nsamp, EbN0_db):
    L = 2**k
    SNR_db = EbN0_db - 10*np.log10(nsamp/(2*k))
    SNR = 10 ** (SNR_db * 0.1)
    x = 2 * np.floor(L * np.random.rand(M)) - L + 1
    P_x = (L**2 - 1) / 3
    Measured_x = np.sum(x**2) / len(x)

    y = []
    for i in range(len(x)):
        y.extend([x[i]] * nsamp)
    y = np.array(y)

    noise = np.random.normal(0, np.sqrt(Measured_x / SNR), len(y))
    y_noisy = y + noise

    y = np.reshape(y_noisy, (M, nsamp))
    matched = np.ones((nsamp, 1))
    z = np.matmul(y, matched) / nsamp
    l = np.arange(-L+1, L, 2)

    z = z[:, 0]
    errors = 0
    for i in range(len(z)):
        differences = np.abs(l - z[i])
        m = np.min(differences)
        index = np.where(differences == m)[0][0]
        z[i] = l[index]
        if x[i] != z[i]:
            errors += 1
    
    return errors


def rootRaisedCosine(nsamp, roll_off, delay):
    F0 = 0.5 / nsamp
    Br = 1
    Fs = Br * nsamp
    Td = 1 / Br
    Ts = 1 / Fs
    F1 = F0 * (1 - roll_off)
    F2 = F0 * (1 + roll_off)
    filter_order = 2 * nsamp * delay

    t = np.arange(0, filter_order, Td)
    h = []
    for i in range(len(t)):
        t_shifted = t[i] - filter_order / 2
        if t_shifted == 0:
            h.append(np.sqrt(2 * F0) *(1 + roll_off * ((4 / np.pi) - 1)))
        elif t_shifted == 1 / 8 / roll_off / F0 or t_shifted == - 1 / 8 / roll_off / F0 :
            h.append((roll_off * np.sqrt(F0)) * ((1 + 2 / np.pi) * np.sin(np.pi / 4 / roll_off) + (1 - 2 / np.pi) * np.cos(np.pi / 4 / roll_off)))
        else:
            factor1 = np.sqrt(2 * F0) / (1 - 64 * roll_off * roll_off * F0 * F0 * t_shifted * t_shifted)
            factor2 = np.sin(2 * np.pi * F1 * t_shifted) / (2 * np.pi * F0 * t_shifted)
            factor3 = (4 * roll_off / np.pi) * np.cos(2 * np.pi * F2 * t_shifted)
            h.append(factor1 * (factor2 + factor3))

    return h


def qam_errors(EbNo, M, roll_off, F1, F2, Br):
    F1 = F1 * 1e6  # Convert MHz to Hz
    F2 = F2 * 1e6  # Convert MHz to Hz
    Br = Br * 1e6  # Convert Mbps to Hz
    W = F2 - F1  # Bandwidth in Hz
    fc = F1 + W / 2  # Carrier frequency
    nsamp = int(np.ceil(2 * F2 / Br)) +7  # Number of samples per symbol

    a = 0.25
    L = int(np.sqrt(M))
    l = np.log2(L)
    k = 2 * l
    Nsymb = 10000
    SNR = EbNo - 10 * np.log10(nsamp / k / 2)  # σε db
    core = [1+1j, 1-1j, -1+1j, -1-1j]
    mapping = core[:]
    if l > 1:
        for j in range(1, int(l)):
            mapping = list(map(lambda x: x + j * 2 * core[0], mapping))
            conj_arr = np.conj(mapping)
            mapping = mapping + conj_arr.tolist()
            conj_arr = -np.conj(mapping)
            mapping = mapping + conj_arr.tolist()

    # Generate random sequence
    x = np.floor(2 * np.random.rand(int(k * Nsymb), 1))
    x_temp = np.reshape(x, (int(len(x) / (k)), int(k)))
    xsym = []

    # Split the list into sublists and put the contents of each sublist
    # into a string so that with the int() command it is converted from binary to decimal
    for i in range(len(x_temp)):
        my_str = ''
        y = x_temp[i]
        for j in range(int(np.log2(M))):
            my_str = my_str + str(int(y[j]))
        a = int(my_str, 2)
        xsym = xsym + [a]

    y = []
    for n in range(len(xsym)):
        y = y + [mapping[xsym[n]]]

    delay = 10
    filtorder = delay * nsamp * 2

    shaping_filter = rootRaisedCosine(nsamp, roll_off, delay)
    ytx = upfirdn([1], y, nsamp)  # upsample
    ytx = np.convolve(ytx, shaping_filter)
    m = np.arange(1, len(ytx) + 1)
    s = np.real(np.multiply(ytx, np.exp(1j * 2 * np.pi * fc * m / nsamp)))

    s_matrix = np.matrix(s)  # transpose
    s_matrix = s_matrix.getH()
    s_list = s_matrix.tolist()
    Ps = 10 * np.log10(np.matmul(s, s_list) / len(s))  # Power of complex signal in dB
    Pn = Ps - SNR

    n = np.sqrt(10**(Pn / 10)) * np.random.randn(1, len(ytx))
    snoisy = s + n

    # receiver
    yrx = 2 * np.multiply(snoisy, np.exp(-1j * 2 * np.pi * fc * m / nsamp))
    yrx = yrx[0, :]
    yrx = np.convolve(yrx, shaping_filter)
    yrx = yrx[::nsamp]  # downsample

    yrx = yrx[2 * delay + 0:len(yrx) - 2 * delay]

    yi = yrx.copy()
    yq = np.imag(yi)
    yi = np.real(yi)

    xrx = []
    q = np.arange(-L + 1, L, 2)

    for n in range(len(yrx)):
        differences = np.abs(q - yi[n])  # Array with the differences of the signal from the levels
        m = min(differences)
        [index], = np.where(differences == m)
        yi[n] = q[index]
        differences = np.abs(q - yq[n])  # Array with the differences of the signal from the levels
        m = min(differences)
        [index], = np.where(differences == m)
        yq[n] = q[index]
    error = 0
    for i in range(len(yrx)):
        if y[i] != yi[i] + yq[i] * 1j:
            error += 1 
    return error / len(x)


# Define the root raised cosine filter function
def rootRaisedCosine1(nsamp, roll_off, delay):
    t = np.arange(-delay, delay + 1 / nsamp, 1 / nsamp)
    h = np.zeros(len(t))
    for i in range(len(t)):
        if t[i] == 0.0:
            h[i] = 1.0 - roll_off + 4 * roll_off / np.pi
        elif roll_off != 0 and t[i] == 1 / (4 * roll_off):
            h[i] = roll_off / np.sqrt(2) * ((1 + 2 / np.pi) * np.sin(np.pi / (4 * roll_off)) + (1 - 2 / np.pi) * np.cos(np.pi / (4 * roll_off)))
        elif roll_off != 0 and t[i] == -1 / (4 * roll_off):
            h[i] = roll_off / np.sqrt(2) * ((1 + 2 / np.pi) * np.sin(np.pi / (4 * roll_off)) + (1 - 2 / np.pi) * np.cos(np.pi / (4 * roll_off)))
        else:
            h[i] = (np.sin(np.pi * t[i] * (1 - roll_off)) + 4 * roll_off * t[i] * np.cos(np.pi * t[i] * (1 + roll_off))) / (np.pi * t[i] * (1 - (4 * roll_off * t[i]) ** 2))
    return h


# Define the BER computation functions
def compute_ber_psk(EbNo_dB, M1):
    EbNo_linear = 10**(EbNo_dB / 10)
    if M1 == 2:  # BPSK
        return 0.5 * scipy.special.erfc(np.sqrt(EbNo_linear))
    else:  # M-PSK
        k = np.log2(M1)
        return (1/4*k) * scipy.special.erfc(np.sqrt(EbNo_linear * k) * np.sin(np.pi / M1))


def psk_errors(EbNo_dB, M1):
    Nsymb = 30000  # Number of symbols
    nsamp = 16  # Samples per symbol
    fc = 4  # Carrier frequency
    rolloff = 0.25
    delay = 10
    SNR_dB = EbNo_dB - 10 * np.log10(nsamp / np.log2(M1))
    shaping_filter = rootRaisedCosine1(nsamp, rolloff, delay)
    filtorder = delay * nsamp * 2

    # Generate random bit stream
    bits1 = np.random.randint(0, M1, Nsymb)

    # Map bits to PSK symbols
    symbols = np.exp(1j * (2 * np.pi * bits1 / M1))

    # Upsample and filter
    ytx1 = upfirdn([1], symbols, nsamp)
    ytx1 = np.convolve(ytx1, shaping_filter, mode='same')
    m1 = np.arange(len(ytx1))
    s1 = np.real(ytx1 * np.exp(1j * 2 * np.pi * fc * m1 / nsamp))

    Ps = np.mean(np.abs(s1)**2)
    SNR_linear = 10**(SNR_dB / 10)
    Pn = Ps / SNR_linear
    if M1 == 2:  # BPSK
        noise = np.sqrt(Pn / 4) * (np.random.randn(len(s1)) + 1j * np.random.randn(len(s1)))
    else:  # M-PSK
        noise = np.sqrt(Pn / 4.5) * (np.random.randn(len(s1)) + 1j * np.random.randn(len(s1)))
    snoisy = s1 + noise

    # Receiver
    yrx1 = snoisy * np.exp(-1j * 2 * np.pi * fc * m1 / nsamp)
    yrx1 = np.convolve(yrx1, shaping_filter, mode='same')
    yrx1 = yrx1[::nsamp]

    # Demodulate symbols
    detected_bits1 = np.angle(yrx1) * M1 / (2 * np.pi)
    detected_bits1 = np.round(detected_bits1) % M1

    # Calculate BER
    bit_errors1 = np.sum(bits1 != detected_bits1)
    ber1 = bit_errors1 / len(bits1)
    return ber1


def fsk_errors_coh(bps, Nsymb, ns, EbNo):
    # Input parameters
    M = 2 ** bps  # number of different symbols
    BR = 1  # Baud Rate
    fc = 2 * M * BR  # RF frequency

    # Derived parameters
    nb = bps * Nsymb  # number of simulated data bits
    T = 1 / BR  # one symbol period
    Ts = T / ns  # oversampling period

    # M frequencies in "coherent" distance (BR)
    f = fc + (BR/2) * (np.arange(1, M + 1) - (M + 1) / 2)

    # Calculate the maximum frequency
    fmax = np.max(f)

    # Recalculate ns to ensure Fs = ns * BR > 2 * fmax
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10

    # Recalculate the oversampling period
    Ts = T / ns

    # awgn channel
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

    # input data bits
    y = np.random.randint(0, 2, nb)
    x = y.reshape((Nsymb, bps))

    t = np.arange(0, len(x) * T, T)  # time vector on the T grid
    tks = np.arange(0, T, Ts)  # oversampling time vector

    # FSK signal
    s = []
    A = np.sqrt(2 / T / ns)
    for k in range(len(x)):
        fk = f[int(''.join(map(str, x[k])), 2)]
        tk = (k * T) + tks
        s.append(np.sin(2 * np.pi * fk * tk))
    s = np.concatenate(s)

    # add noise to the FSK (passband) signal
    s = awgn(s, SNR)

    # FSK receiver
    xr = []
    for k in range(len(s) // ns):
        tk = (k * T) + tks
        sk = s[k * ns:(k + 1) * ns]
        smi = []
        for fi in f:
            si = np.sin(2 * np.pi * fi * tk)
            smi.append(np.sum(sk * si))
        j = np.argmax(smi)
        xr.append([int(bit) for bit in bin(j)[2:].zfill(bps)])
    xr = np.array(xr).reshape((Nsymb, bps))

    # count errors
    errors = np.sum(x != xr)
    return errors


def fsk_errors_non_coh(bps, Nsymb, ns, EbNo):
    M = 2 ** bps  # number of different symbols
    BR = 1  # Baud Rate
    fc = 2 * M * BR  # RF frequency

    # Derived parameters
    nb = bps * Nsymb  # number of simulated data bits
    T = 1 / BR  # one symbol period
    Ts = T / ns  # oversampling period
    f = fc + BR * (np.arange(1, M + 1) - (M + 1) / 2)  # M frequencies

    # Calculate the maximum frequency
    fmax = np.max(f)

    # Recalculate ns to ensure Fs = ns * BR > 2 * fmax
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10

    # Recalculate the oversampling period
    Ts = T / ns

    # AWGN channel
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

    # input data bits
    y = np.random.randint(0, 2, nb)  # Ensure integer dimensions
    x = y.reshape((-1, bps))
    t = np.arange(0, len(x) * T, T)  # time vector on the T grid
    tks = np.arange(0, T, Ts)

    # FSK signal
    s = np.array([])
    A = np.sqrt(2 / T / ns)
    for k in range(len(x)):
        fk = f[int("".join(map(str, x[k])), 2)]
        tk = (k * T) + tks
        s = np.concatenate((s, np.sin(2 * np.pi * fk * tk)))

    # add noise to the FSK (passband) signal
    s = awgn(s, SNR)

    # FSK receiver
    # Non-coherent demodulation
    xr = np.array([])
    for k in range(len(s) // ns):
        tk = (k * T) + tks
        sk = s[k * ns:(k + 1) * ns]
        sm = []
        for i in range(M):
            si = np.sin(2 * np.pi * (f[i] * tk))
            sq = np.cos(2 * np.pi * (f[i] * tk))
            smi = np.sum(sk * si[:len(sk)])
            smq = np.sum(sk * sq[:len(sk)])
            sm.append(np.sqrt(smi ** 2 + smq ** 2))
        j = np.argmax(sm)
        xr = np.concatenate((xr, np.array(list(np.binary_repr(j, width=bps)), dtype=int)))
    # count errors
    x_reshaped = x.reshape(-1)
    xr_reshaped = xr.reshape(-1)
    err = np.not_equal(x_reshaped, xr_reshaped[:len(x_reshaped)])
    errors = np.sum(err)
    return errors


# Define the BER values directly inside the functions
def theoretical_ber_coh(EbNo, M):
    if M == 2:
        return np.array([0.15866, 0.13093, 0.10403, 0.078896, 0.056495,
    0.037679, 0.023007, 0.012587, 0.0060044, 0.0024133,
    0.0007827, 0.00019399, 3.4303e-05, 3.9692e-06, 2.6951e-07])
    elif M == 4:
        return np.array([0.11814, 0.087789, 0.060786, 0.038512, 0.021824, 
    0.010751, 0.0044428, 0.0014733, 0.00037102, 6.6229e-05, 
    7.6892e-06, 5.2118e-07, 1.7997e-08, 2.6653e-10, 1.362e-12])
    elif M == 8:
        return np.array([0.10227, 0.067834, 0.041318, 0.022024, 0.0099156, 
    0.0036087, 0.0010058, 0.00020086, 2.6486e-05, 2.0836e-06, 
    8.6119e-08, 1.5924e-09, 1.074e-11, 2.0391e-14, 7.8524e-18])
    elif M == 16:
        return np.array([0.089859, 0.055663, 0.029957, 0.013469, 0.004819,
    0.001293, 0.00024205, 2.897e-05, 1.9921e-06, 6.8928e-08,
    1.0145e-09, 5.1257e-12, 6.7629e-15, 1.6453e-18, 4.6157e-23])
    elif M == 32:
        return np.array([0.082719, 0.047105, 0.022469, 0.0085348, 0.0024266,
    0.00047917, 6.0083e-05, 4.2975e-06, 1.5388e-07, 2.3425e-09,
    1.2294e-11, 1.6992e-14, 4.3826e-18, 1.3266e-22, 2.1688e-28])
    else:
        raise ValueError("Unsupported value of M for coherent case")


def theoretical_ber_non_coh(EbNo, M):
    if M == 2:
        return np.array([0.30327, 0.26644, 0.22637, 0.18438, 0.1424, 0.10287, 
    0.068311, 0.0408, 0.021324, 0.0094212, 0.003369, 
    0.0009231, 0.00018089, 2.3244e-05, 1.7558e-06])
    elif M == 4:
        return np.array([0.22934, 0.18475, 0.13987, 0.097719, 0.061557, 
    0.033946, 0.01579, 0.0059139, 0.0016837, 0.00033939, 
    4.4371e-05, 3.3753e-06, 1.3045e-07, 2.1593e-09, 1.233e-11])
    elif M == 8:
        return np.array([0.19472, 0.14559, 0.099187, 0.059806, 0.030757, 
    0.012878, 0.0041438, 0.00095467, 0.00014449, 1.2945e-05, 
    6.0428e-07, 1.2541e-08, 9.4638e-11, 2.0092e-13, 8.6607e-17])
    elif M == 16:
        return np.array([0.17469, 0.12169, 0.074737, 0.038861, 0.01625, 
    0.005127, 0.0011288, 0.00015786, 1.2538e-05, 4.943e-07, 
    8.2001e-09, 4.6432e-11, 6.8517e-14, 1.8682e-17, 6.0826e-22])
    elif M == 32:
        return np.array([0.16103, 0.10471, 0.058014, 0.025984, 0.0088058, 
    0.0020817, 0.00031127, 2.6219e-05, 1.0859e-06, 1.8789e-08, 
    1.1086e-10, 1.7154e-13, 4.9582e-17, 1.737e-21, 4.2722e-27])
    else:
        raise ValueError("Unsupported value of M for non-coherent case")


def msk_errors(M, nsamp, EbN0_db, precoding=True):
    Nbits = 10000  # number of data bits
    n = Nbits
    R = 2000000  # bit rate
    fc = 8000000  # carrier frequency
    ns = nsamp  # oversampling factor
    EbNo = EbN0_db
    result = 0

    if (precoding):
        # AWGN channel
        SNR = EbNo - 10 * np.log10(ns/2)  # in dB
        T = 1 / R  # 1-bit period (= basic period)
        Ts = T / ns  # sampling frequency
        fss = 1/Ts

        # Input sequence
        y = np.concatenate(([1], np.sign(np.random.rand(n - 1) - 0.5)))  # random numbers, -1 or 1
        x = y

        g = np.ones(ns)
        xx = upfirdn(g, x, up=ns)  # NRZ polar pulse train samples

        # Time grid
        ts = np.arange(0, len(xx) * Ts, Ts)  # of length ns*(n+1)

        ## MSK TRANSMITTER
        xs = xx
        theta = np.cumsum(xs) * np.pi / 2 / ns
        xs_i = np.cos(theta)  # in-phase component
        xs_q = np.sin(theta)  # quadrature component

        # Ensure that xs_i and xs_q are the same length as the time grid `ts`
        if len(xs_i) > len(ts):
            xs_i = xs_i[:len(ts)]
            xs_q = xs_q[:len(ts)]
        elif len(ts) > len(xs_i):
            ts = ts[:len(xs_i)]

        # Modulation
        s = xs_i * np.cos(2 * np.pi * fc * ts) - xs_q * np.sin(2 * np.pi * fc * ts)

        # Addition of noise
        s = awgn(s, SNR)

        ## MSK RECEIVER
        xs_i = s * np.cos(2 * np.pi * fc * ts)
        xs_q = -s * np.sin(2 * np.pi * fc * ts)

        # LP (Parks-McClellan) filter
        f1 = 0.75*(fss/2)/ns
        f2 = 4*f1
        order = 8 * ns
        fpts = [0, f1, f2, fss/2]
        mag = [1, 1, 0, 0]
        wt = [1, 1]
        b = firwin2(order+1, fpts, mag, fs=fss)
        a = 1

        len_xs_i = len(xs_i)
        dummy = np.concatenate((xs_i, np.zeros(order)))
        dummy1 = lfilter(b, a, dummy)
        delay = order // 2
        xs_i = dummy1[delay:delay + len_xs_i]
        xs_i = np.concatenate((xs_i, np.ones(nsamp-1)))

        dummy = np.concatenate((xs_q, np.zeros(order)))
        dummy1 = lfilter(b, a, dummy)
        xs_q = dummy1[delay:delay + len_xs_i]

        bi = 1
        xr_1 = 1
        xr = np.zeros(n)
        for k in range(0, n, 2):
            li = np.arange((k+1) * ns, min((k + 3) * ns-1, len(xs_i)))
            lq = np.arange(k * ns, min((k + 2) * ns-1, len(xs_q)))
            xi = xs_i[li]
            xq = xs_q[lq]
            gmi = np.cos(np.pi / 2 / T * Ts * li)  # matched-filter pulse
            gmq = -gmi  # =sin(pi/2/T*Ts*lq);
            bi_1 = bi
            bi = np.sign(np.sum(xi * gmi))
            bq = np.sign(np.sum(xq * gmq))
            xr[k] = bi_1 * bq
            xr[k+1] = bi * bq
            xr_1 = xr[k + 1]

        xr = xr.reshape(-1)
        err = np.not_equal(x, xr)
        errors = np.sum(err)
        result = errors / Nbits

    else:
        # AWGN channel
        SNR = EbNo - 10 * np.log10(ns/2)  # in dB
        T = 1 / R  # 1-bit period (= basic period)
        Ts = T / ns  # sampling frequency
        fss = 1/Ts

        # Input sequence
        y = np.concatenate(([1], np.sign(np.random.rand(n - 1) - 0.5)))  # random numbers, -1 or 1
        x = y
        x[0] = 1
        for i in range(1, len(y)):
            x[i] = y[i] * x[i-1]  # Apply precoding rule

            

        g = np.ones(ns)
        xx = upfirdn(g, x, up=ns)  # NRZ polar pulse train samples

        # Time grid
        ts = np.arange(0, len(xx) * Ts, Ts)  # of length ns*(n+1)

        ## MSK TRANSMITTER
        xs = xx
        theta = np.cumsum(xs) * np.pi / 2 / ns
        xs_i = np.cos(theta)  # in-phase component
        xs_q = np.sin(theta)  # quadrature component

        # Ensure that xs_i and xs_q are the same length as the time grid ts
        if len(xs_i) > len(ts):
            xs_i = xs_i[:len(ts)]
            xs_q = xs_q[:len(ts)]
        elif len(ts) > len(xs_i):
            ts = ts[:len(xs_i)]

        # Modulation
        s = xs_i * np.cos(2 * np.pi * fc * ts) - xs_q * np.sin(2 * np.pi * fc * ts)

        # Addition of noise
        s = awgn(s, SNR)

        ## MSK RECEIVER
        xs_i = s * np.cos(2 * np.pi * fc * ts)
        xs_q = -s * np.sin(2 * np.pi * fc * ts)

        # LP (Parks-McClellan) filter
        f1 = 0.75*(fss/2)/ns
        f2 = 4*f1
        order = 8 * ns
        fpts = [0, f1, f2, fss/2]
        mag = [1, 1, 0, 0]
        wt = [1, 1]
        b = firwin2(order+1, fpts, mag, fs=fss)
        a = 1

        len_xs_i = len(xs_i)
        dummy = np.concatenate((xs_i, np.zeros(order)))
        dummy1 = lfilter(b, a, dummy)
        delay = order // 2
        xs_i = dummy1[delay:delay + len_xs_i]
        xs_i = np.concatenate((xs_i, np.ones(nsamp-1)))

        dummy = np.concatenate((xs_q, np.zeros(order)))
        dummy1 = lfilter(b, a, dummy)
        xs_q = dummy1[delay:delay + len_xs_i]

        # Updated MSK decoding for precoded bits with recursive logic
        bi = 1
        xr_1 = 1  # Initialize previous decoded bit (xr_1)
        xr = np.zeros(n)  # Array to store decoded bits

        for k in range(0, n, 2):
            li = np.arange((k+1) * ns, min((k + 3) * ns-1, len(xs_i)))
            lq = np.arange(k * ns, min((k + 2) * ns-1, len(xs_q)))
            xi = xs_i[li]
            xq = xs_q[lq]
            
            # Matched filter output (to match MSK modulation characteristics)
            gmi = np.cos(np.pi / 2 / T * Ts * li)  # In-phase matched filter pulse
            gmq = -gmi  # Quadrature matched-filter pulse (sin is negative of cosine)
            
            # Save previous in-phase matched filter output
            bi_1 = bi
            
            # Decode in-phase (I) and quadrature (Q) components
            bi = np.sign(np.sum(xi * gmi))
            bq = np.sign(np.sum(xq * gmq))
            
            # Apply recursive decoding rule for precoded MSK
            xr[k] = bi_1 * bq  # Decode the k-th bit
            xr[k+1] = bi * bq  # Decode the (k+1)-th bit
            
            # Update the previously decoded bit (xr_1)
            xr_1 = xr[k+1]

        xr = xr.reshape(-1)
        err = np.not_equal(y, xr)
        errors = np.sum(err)
        result = 0.5*errors / Nbits

    return result


M = 2000
nsamp = 16
EbN0_db = np.arange(1, 15, 2) 
EbN0 = 10 ** (EbN0_db / 10)
EbNo_dB_theory = np.arange(0, 15, 1)  # Eb/No values in dB for theoretical
EbNo_theory = 10 ** (EbNo_dB_theory / 10)  # convert dB to linear for theoretical

def plot_selected_modulations(change):
    # Start the timer
    loader_html3.value = loading
    start_time = time.time()

    with plot_output:
        loader_html3.value = loading
        plot_output.clear_output(wait=True)
        plt.figure(figsize=(10, 7))
        
        modulation = modulation_dropdown.value
        level = levels_dropdown.value
        if level is None:
            return
        if modulation == 'MSK':
            L = 2
        else:
            L = int(level.split('-')[0])
        L1 = int(np.sqrt(L))
        k = int(np.log2(L))
        modulation_name = level
        
        if modulation == 'ASK':
            ber = [ask_errors(k, M, nsamp, db) / M / np.log2(L) for db in EbN0_db]
            ber_theoretical = (((L - 1) / L) * scipy.special.erfc(np.sqrt(EbN0 * (3 * np.log2(L)) / (L**2 - 1)))) / k
        elif modulation == 'QAM':
            roll_off = roll_off_input.value
            F1 = 6.75
            F2 = 9.25
            Br = Br_input.value
            ber = [qam_errors(db, L, roll_off, F1, F2, Br) for db in EbN0_db]
            ber_theoretical = [((L1 - 1) /(L1*np.log2(L1)) * scipy.special.erfc(np.sqrt(3 * np.log2(L1) / (L1 * L1 - 1) * 10**(db/10)))) for db in EbN0_db]
        elif modulation == 'PSK':
            ber = [psk_errors(db, L) for db in EbN0_db]
            ber_theoretical = [compute_ber_psk(db, L) for db in EbN0_db]
        elif modulation == 'FSK':
            ber_coh = [fsk_errors_coh(k, M, nsamp, db) / M / np.log2(L) for db in EbN0_db]
            ber_non_coh = [fsk_errors_non_coh(k, M, nsamp, db) / M / np.log2(L) for db in EbN0_db]
            ber_coh_theoretical = theoretical_ber_coh(EbNo_theory, L)
            ber_non_coh_theoretical = theoretical_ber_non_coh(EbNo_theory, L)
            plt.semilogy(EbN0_db, ber_coh, 'o', label=f'Experimental Coherent {modulation_name}', color='#1F77B4')
            plt.semilogy(EbN0_db, ber_non_coh, 'o', label=f'Experimental Non-Coherent {modulation_name}', color='#FF7F0E')
        elif modulation == 'MSK':
            EbNo_range = np.arange(0, 9, 1)
            ber = [msk_errors(M, nsamp, db, precoding=True) for db in EbNo_range]
            if levels_dropdown.value == 'MSK (with precoding)':
                ber_theoretical = [(erfc(np.sqrt(10**(db / 10)))) for db in EbNo_range]
            else:
                ber_theoretical = [(0.9 * erfc(np.sqrt(10**(db / 10)))) for db in EbNo_range]
        
        if modulation == 'MSK':
            EbNo_range = np.arange(0, 9, 1)
            plt.semilogy(EbNo_range, ber_theoretical, linestyle='-', label=f'Theoretical {modulation_name}')
            plt.semilogy(EbNo_range, ber, 'o', label=f'Experimental {modulation_name}')
        elif modulation != 'FSK':
            plt.semilogy(EbN0_db, ber_theoretical, linestyle='-', label=f'Theoretical {modulation_name}')
            plt.semilogy(EbN0_db, ber, 'o', label=f'Experimental {modulation_name}')
        else:
            plt.semilogy(EbNo_dB_theory, ber_coh_theoretical, linestyle='-', label=f'Theoretical Coherent {modulation_name}')
            plt.semilogy(EbNo_dB_theory, ber_non_coh_theoretical, linestyle='-', label=f'Theoretical Non-Coherent {modulation_name}')

        plt.grid(True, which='both')
        plt.xlabel("Eb/N0 (dB)")
        plt.ylabel("Bit Error Rate")
        plt.legend()
        plt.title(f'{modulation_name} | Theoretical and Experimental BER')

        # Show elapsed time
        elapsed_time = time.time() - start_time
        timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
        loader_html3.value = done

        plt.show()


# Define additional input boxes for roll-off, F1, F2, and Br
roll_off_input = widgets.FloatText(value=0.25, description='Roll-off:')
F1_input = widgets.FloatText(value=6.75, description='F1: (MHz)')
F2_input = widgets.FloatText(value=9.25, description='F2: (MHz)')
Br_input = widgets.FloatText(value=10, description='Br: (Mbps)')

# Container for QAM specific inputs
qam_inputs = widgets.VBox([roll_off_input, Br_input])

# Define the plot button
plot_button = widgets.Button(description="Plot", button_style='primary')

# Attach the plot_selected_modulations function to the 'click' event of the button
plot_button.on_click(plot_selected_modulations)

inputs = widgets.VBox([modulation_dropdown, levels_dropdown])

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

inputs_ui = widgets.HBox([inputs, loader_timer_box], layout=Layout(align_items='center'))

ui = widgets.VBox([inputs_ui, plot_button]) 

# Update the UI to include QAM specific inputs conditionally
def update_ui(change):
    if modulation_dropdown.value == 'QAM':
        inputs_ui.children = [inputs, qam_inputs, loader_timer_box]
    else:
        inputs_ui.children = [inputs, loader_timer_box]

# Attach the update_ui function to the 'value' property of the modulation dropdown
modulation_dropdown.observe(update_ui, names='value')

# Call the update_ui function initially to set the correct UI state
update_ui(None)

# Setup the display layout
display(ui, plot_output)

# Display the initial plot
plot_selected_modulations(None)


Output()